In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Build a Flask API

In [ ]:
# Load trained model
model = joblib.load("best_model_XGBoost.pkl")

app = Flask(__name__)

@app.route("/")
def home():
    return {"message": "Model API is running!"}

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()  # Expecting JSON input
    df = pd.DataFrame([data])  # Convert dict to DataFrame

    prediction = model.predict(df)[0]
    probability = model.predict_proba(df)[0].tolist()

    return jsonify({
        "prediction": int(prediction),
        "probability": probability
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

### Containerize with Docker

In [ ]:
FROM python:3.9-slim

# Set working dir
WORKDIR /app

# Copy project files
COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

COPY . .

# Run Flask
CMD ["python", "app.py"]


flask
pandas
joblib
scikit-learn
xgboost


docker build -t mymodel-api .
docker run -p 5000:5000 mymodel-api



### Connect to BI

In [ ]:
let
    url = "http://localhost:5000/predict",
    body = "{""feature1"": 10, ""feature2"": 5, ""feature3"": ""A""}",
    Source = Json.Document(Web.Contents(url, [
        Content = Text.ToBinary(body),
        Headers = [#"Content-Type"="application/json"],
        ManualStatusHandling={400,404,500}
    ]))
in
    Source
